In [1]:
import pandas as pd
import numpy as np

In [2]:
target = pd.read_csv("MT2017.csv")
league = pd.read_csv("MLB2017.csv")
injury_date = (5.06, 5.28)
recovered_date = (5.10,7.09)
strong_start = (0,)
strong_end = (0,)
min_to_be_Strong = 162
min_to_be_Average = 162

In [3]:
target = target[['Date', 'Tm', 'Opp', 'W/L']]
league = league[['Tm', 'W']]
league =league.rename(columns={'Tm' : 'Opp'})

In [4]:
league.drop(league.tail(1).index,inplace=True) # drop last 1 rows

In [5]:
target[target['Opp'] == 'MIN']

,Date,Tm,Opp,W/L
56,Thursday Jun 1,LAA,MIN,L
57,Friday Jun 2,LAA,MIN,L
58,Saturday Jun 3,LAA,MIN,W
59,Sunday Jun 4,LAA,MIN,L
86,Monday Jul 3,LAA,MIN,L
87,Tuesday Jul 4,LAA,MIN,L
88,Wednesday Jul 5,LAA,MIN,W


In [6]:
new_win = []
new_total = []
pro = []
for i in league.index:
    new_win.append(league['W'][i] - len(target[ (target['Opp' ]== league['Opp'][i]) & (target['W/L'].str.contains("L")) ]))
    #new_total.append( 162-len(target[ (target['Opp']== league['Opp'][i])] )
    pro.append(  round( 162 *     (league['W'][i] - len(target[ (target['Opp' ]== league['Opp'][i]) & (target['W/L'].str.contains("L")) ]) )      /      (162-len(target[ (target['Opp' ]== league['Opp'][i])]) ) ) )
league['Isolated_W'] = new_win 
#league['Isolated_All'] = new_total
league['probability'] = pro

In [7]:
league = league.sort_values(by = ['probability'])
league = league.reset_index(drop=True)
league

,Opp,W,Isolated_W,probability
0,DET,64,61,64.0
1,SFG,64,64,64.0
2,CHW,67,64,67.0
3,PHI,66,66,67.0
4,CIN,68,68,68.0
5,NYM,70,68,69.0
6,SDP,71,71,71.0
7,ATL,72,71,72.0
8,PIT,75,75,75.0
9,MIA,77,75,76.0


In [8]:
print(league['probability'][9])
print(league['probability'][20])


76.0
86.0


In [9]:
#helper function to modify the date data for later analysis
def modi_date(date):
    date_dict = {
        'Mar': '3',
        'Apr': '4',
        'May': '5',
        'Jun': '6',
        'Jul': '7',
        'Aug': '8',
        'Sep': '9',
        'Oct': '10'
    }
    date_split = date.split(" ")[1:]
    d = date_dict[date_split[0]]
    return float('{}.{:0>2}'.format(date_dict[date_split[0]], date_split[1]))

In [10]:
df = pd.merge(target, league, on = 'Opp')
df['Date'] = df['Date'].apply(modi_date)
df = df.sort_values(by = ['Date'])
df = df.reset_index(drop=True)

In [11]:
def get_winning_games_number(team_name: str):
    return league[league['Opp'] == team_name]['probability'].values[0]

In [12]:
#helper function to get the classify an opponent's status: weak, average, or strong
def get_oppo_status(team_name: str):
    w_n = get_winning_games_number(team_name)
    if w_n >= min_to_be_Strong:
        return "Strong"
    elif min_to_be_Average <= w_n < min_to_be_Strong:
        return 'Average'
    else:
        return "Weak"

In [13]:
def get_avaliablity(date):
    for inj, rec in zip(injury_date, recovered_date):
        if inj <= date <= rec:
            return False
    return True

In [14]:
def is_Strong(date):
    for start, end in zip(strong_start, strong_end):
        if start <= date < end:
            return True
    return False

In [15]:
frame = pd.DataFrame({
                      'Date': df['Date'], 
                      "Aval": df['Date'].apply(get_avaliablity), 
                      "Strong": df['Date'].apply(is_Strong),
                      "Opp-Status": df['Opp'].apply(get_oppo_status),
                      'Opp': df['Opp'],
                      'W/L': df['W/L'], 
                      'Opp-W': df['Isolated_W'],
                      'Opp-W-percentage': df['probability']
                      })
frame.to_excel('MT2017.xlsx')

In [16]:
#helper funtion to calculate the number of the games in which the target team and its
#opponent are classifies as a particular status.
def count_category(data, strong=False, opp_status=None):
    strong_series = data['Strong'] if strong else data['Strong'] == False
    c1 = len(data[strong_series & (frame['Opp-Status'].str.match(opp_status))])
    a1 = len(data)
    #print('numerator :    ', c1)
    #print('denumerator : ', a1)
    #print('output: ', c1/a1)
    return c1/a1

In [17]:
def count_win(data, aval=False, strong=False, opp_status=None):
    aval_series = data['Aval'] if aval else data['Aval'] == False
    strong_series = data['Strong'] if strong else data['Strong'] == False
    win = len(data[(data['W/L'].str.contains("W")) & aval_series & strong_series & (frame['Opp-Status'].str.match(opp_status))])
    number_of_all_games =  len(data[aval_series & strong_series & (frame['Opp-Status'].str.match(opp_status))])
    #print('numerator :    ', win)
    #print('denumerator : ', number_of_all_games)
    if number_of_all_games == 0:
        #print('output: ', 0)
        return 0
    else: 
        #print('output: ', win/number_of_all_games)
        return win/number_of_all_games

In [18]:
def get_WAR(data = frame):
    
    #print('P_Aval_Weak_weakOpp :')
    aval_weak_weak = count_win(data, aval = True, strong = False, opp_status="Weak") 
    #print('P_Unaval_Weak_weakOpp :')
    unaval_weal_weak = count_win(data, aval = False, strong = False, opp_status="Weak")
    #print('P X = Weak_weakOpp :')
    weak_weak = count_category(data, strong = False, opp_status="Weak")
    
    #print('P_Aval_Weak_averageOpp :')
    aval_weak_average = count_win(data, aval = True, strong = False, opp_status="Average")
    #print('P_Unaval_Weak_averageOpp :')
    unaval_weak_average = count_win(data, aval = False, strong = False, opp_status="Average")
    #print('P X = Weak_averageOpp :')
    weak_average = count_category(data, strong = False, opp_status="Average")
    
    #print('P_Aval_Weak_strongOpp :')
    aval_weak_strong = count_win(data, aval = True, strong = False, opp_status="Strong")
    #print('P_Unaval_Weak_strongOpp :')
    unaval_weak_strong = count_win(data, aval = False, strong = False, opp_status="Strong")
    #print('P X = Weak_strongOpp :')
    weak_strong = count_category(data, strong = False, opp_status="Strong")
    
    #print('P_Aval_Strong_weakOpp :')
    aval_strong_weak = count_win(data, aval = True, strong = True, opp_status="Weak") 
    #print('P_Unaval_Strong_weakOpp :')
    unaval_strong_weak = count_win(data, aval = False, strong = True, opp_status="Weak")
    #print('P X = Strong_weakOpp :')
    strong_weak = count_category(data, strong = True, opp_status="Weak")
    
    #print('P_Aval_Strong_averageOpp :')
    aval_strong_average = count_win(data, aval = True, strong = True, opp_status="Average")
    #print('P_Unaval_Strong_averageOpp :')
    unaval_strong_average = count_win(data, aval = False, strong = True, opp_status="Average")
    #print('P X = Strong_averageOpp :')
    strong_average = count_category(data, strong = True, opp_status="Average")
    
    #print('P_Aval_Strong_strongOpp :')
    aval_strong_strong = count_win(data, aval = True, strong = True, opp_status="Strong")
    #print('P_Unaval_Strong_strongOpp :')
    unaval_strong_strong = count_win(data, aval = False, strong = True, opp_status="Strong")
    #print('P X = Strong_strongOpp :')
    strong_strong = count_category(data, strong = True, opp_status="Strong")
    
    war = ((aval_weak_weak - unaval_weal_weak)* weak_weak
           + (aval_weak_average - unaval_weak_average)* weak_average
           + (aval_weak_strong - unaval_weak_strong)* weak_strong 
           + (aval_strong_weak - unaval_strong_weak)* strong_weak
           + (aval_strong_average - unaval_strong_average)* strong_average 
           + (aval_strong_strong - unaval_strong_strong)* strong_strong) * 119
    return war

In [19]:
def WAR_change_oppSAW():
    output = []
    new_df = frame
    for i in range(80,95):
        for j in range(70, i):
            
            #helper function to change the data
            def new_opp_status(team_name: str):
                w_n = get_winning_games_number(team_name)
                if w_n >= i:
                    return "Strong"
                elif j <= w_n < i:
                    return 'Average'
                else:
                    return "Weak"
                
            new_df['Opp-Status'] = new_df['Opp'].apply(new_opp_status)
            twar = round(get_WAR(new_df), 4)
            temp = [i, j, twar]
            output.append(temp)
    return output

In [20]:
x = WAR_change_oppSAW()

In [21]:
x

[[80, 70, 4.6386],
 [80, 71, 4.6386],
 [80, 72, 4.6386],
 [80, 73, 8.3096],
 [80, 74, 8.3096],
 [80, 75, 8.3096],
 [80, 76, 8.3096],
 [80, 77, 4.8698],
 [80, 78, 7.8392],
 [80, 79, 7.8392],
 [81, 70, 4.1004],
 [81, 71, 4.1004],
 [81, 72, 4.1004],
 [81, 73, 7.7639],
 [81, 74, 7.7639],
 [81, 75, 7.7639],
 [81, 76, 7.7639],
 [81, 77, 4.5076],
 [81, 78, 5.3819],
 [81, 79, 5.3819],
 [81, 80, 5.0062],
 [82, 70, 4.1004],
 [82, 71, 4.1004],
 [82, 72, 4.1004],
 [82, 73, 7.7639],
 [82, 74, 7.7639],
 [82, 75, 7.7639],
 [82, 76, 7.7639],
 [82, 77, 4.5076],
 [82, 78, 5.3819],
 [82, 79, 5.3819],
 [82, 80, 5.0062],
 [82, 81, 4.9221],
 [83, 70, 4.1004],
 [83, 71, 4.1004],
 [83, 72, 4.1004],
 [83, 73, 7.7639],
 [83, 74, 7.7639],
 [83, 75, 7.7639],
 [83, 76, 7.7639],
 [83, 77, 4.5076],
 [83, 78, 5.3819],
 [83, 79, 5.3819],
 [83, 80, 5.0062],
 [83, 81, 4.9221],
 [83, 82, 4.9221],
 [84, 70, 4.1004],
 [84, 71, 4.1004],
 [84, 72, 4.1004],
 [84, 73, 7.7639],
 [84, 74, 7.7639],
 [84, 75, 7.7639],
 [84, 76, 7.

In [22]:
factor1 = []
factor2 = []
result = []
for row in x:
    factor1.append(row[0])
    factor2.append(row[1])
    result.append(row[2])
df2 = pd.DataFrame({
    'min_to_be_Strong': factor1,
    'min_to_be_Average': factor2,
    'war': result
    })
df2 = df2.pivot(index='min_to_be_Average', columns='min_to_be_Strong', values='war')
df2.to_excel('MT2017_WAR.xlsx')

In [23]:
df2

min_to_be_Strong,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94
min_to_be_Average,,,,,,,,,,,,,,,
70,4.6386,4.1004,4.1004,4.1004,4.1004,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,2.8547,2.8547
71,4.6386,4.1004,4.1004,4.1004,4.1004,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,2.8547,2.8547
72,4.6386,4.1004,4.1004,4.1004,4.1004,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,4.7092,2.8547,2.8547
73,8.3096,7.7639,7.7639,7.7639,7.7639,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,4.5022,4.5022
74,8.3096,7.7639,7.7639,7.7639,7.7639,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,4.5022,4.5022
75,8.3096,7.7639,7.7639,7.7639,7.7639,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,4.5022,4.5022
76,8.3096,7.7639,7.7639,7.7639,7.7639,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,7.3213,4.5022,4.5022
77,4.8698,4.5076,4.5076,4.5076,4.5076,4.4192,4.4192,4.4192,4.4192,4.4192,4.4192,4.4192,4.4192,3.2893,3.2893
78,7.8392,5.3819,5.3819,5.3819,5.3819,5.7444,5.7444,5.7444,5.7444,5.7444,5.7444,5.7444,5.7444,5.3491,5.3491
